In [5]:
import numpy as np

class FuzzyVariable:
    def __init__(self, name, ranges):
        self.name = name
        self.ranges = ranges

    def fuzzify(self, value):
        memberships = {}
        for term, (a, b, c, d) in self.ranges.items():
            if a <= value <= b:
                memberships[term] = (value - a) / (b - a)
            elif b <= value <= c:
                memberships[term] = 1
            elif c <= value <= d:
                memberships[term] = (d - value) / (d - c)
            else:
                memberships[term] = 0
        return memberships

class FuzzyRule:
    def __init__(self, conditions, output):
        self.conditions = conditions
        self.output = output

    def evaluate(self, memberships):
        condition_values = [memberships[var][term] for var, term in self.conditions.items()]
        return min(condition_values)

class FuzzyInferenceSystem:
    def __init__(self, variables, rules):
        self.variables = variables
        self.rules = rules

    def infer(self, inputs):
        fuzzy_values = {var: self.variables[var].fuzzify(value) for var, value in inputs.items()}
        outputs = {}
        for rule in self.rules:
            rule_strength = rule.evaluate(fuzzy_values)
            output_term, output_range = rule.output
            if output_term not in outputs:
                outputs[output_term] = []
            outputs[output_term].append((rule_strength, output_range))

        aggregated = {}
        for term, values in outputs.items():
            total_strength = sum(strength for strength, _ in values)
            if total_strength > 0:
                aggregated[term] = sum(strength * np.mean(output_range) for strength, output_range in values) / total_strength
            else:
                aggregated[term] = 0
        return aggregated

# Defining fuzzy variables
permintaan = FuzzyVariable("Permintaan", {
    "turun": (0, 0, 2000, 4000),
    "tetap": (2000, 4000, 4000, 6000),
    "naik": (4000, 6000, 7000, 7000)
})

persediaan = FuzzyVariable("Persediaan", {
    "sedikit": (0, 0, 200, 400),
    "sedang": (200, 400, 400, 800),
    "banyak": (400, 800, 1000, 1000)
})

produksi = FuzzyVariable("Produksi", {
    "berkurang": (0, 0, 2000, 4000),
    "bertambah": (2000, 4000, 8000, 8000)
})

# Defining fuzzy rules
rules = [
    FuzzyRule({"Permintaan": "turun", "Persediaan": "sedikit"}, ("bertambah", (6000, 8000))),
    FuzzyRule({"Permintaan": "turun", "Persediaan": "sedang"}, ("bertambah", (4000, 6000))),
    FuzzyRule({"Permintaan": "turun", "Persediaan": "banyak"}, ("berkurang", (2000, 4000))),
    FuzzyRule({"Permintaan": "tetap", "Persediaan": "sedikit"}, ("bertambah", (4000, 6000))),
    FuzzyRule({"Permintaan": "tetap", "Persediaan": "banyak"}, ("berkurang", (2000, 4000))),
    FuzzyRule({"Permintaan": "naik", "Persediaan": "sedikit"}, ("bertambah", (6000, 8000))),
    FuzzyRule({"Permintaan": "naik", "Persediaan": "banyak"}, ("berkurang", (2000, 4000)))
]

# Creating Fuzzy Inference System
fis = FuzzyInferenceSystem({"Permintaan": permintaan, "Persediaan": persediaan}, rules)

# Example input sets
example_inputs = [
    {"Permintaan": 3500, "Persediaan": 300},
    {"Permintaan": 1500, "Persediaan": 100},
    {"Permintaan": 5000, "Persediaan": 700},
    {"Permintaan": 2500, "Persediaan": 500},
    {"Permintaan": 6000, "Persediaan": 200}
]

# Evaluate and print outputs for all examples
for i, inputs in enumerate(example_inputs):
    output = fis.infer(inputs)
    formatted_output = {k: float(v) for k, v in output.items()}  # Convert np.float64 to float
    print(f"Example {i + 1} Inputs: {inputs} => Output: {formatted_output}")


Example 1 Inputs: {'Permintaan': 3500, 'Persediaan': 300} => Output: {'bertambah': 5500.0, 'berkurang': 0.0}
Example 2 Inputs: {'Permintaan': 1500, 'Persediaan': 100} => Output: {'bertambah': 7000.0, 'berkurang': 0.0}
Example 3 Inputs: {'Permintaan': 5000, 'Persediaan': 700} => Output: {'bertambah': 0.0, 'berkurang': 3000.0}
Example 4 Inputs: {'Permintaan': 2500, 'Persediaan': 500} => Output: {'bertambah': 5000.0, 'berkurang': 3000.0}
Example 5 Inputs: {'Permintaan': 6000, 'Persediaan': 200} => Output: {'bertambah': 7000.0, 'berkurang': 0.0}
